In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from ast import literal_eval
import statistics
import scipy.stats as sp

In [ ]:
df = pd.read_csv("ms_results/PRS_calculations_v4_discovery_0.7_nobootstrap", sep=" ", header=None, converters={9: literal_eval, 10: literal_eval, 11: literal_eval, 12: literal_eval}) 
phen_list = ['phenotype']
anc_list = df[2].unique().tolist()
anc_list = ['African', 'WHG', 'EHG', 'CHG', 'Yamnaya','Farmer','EastAsian']
anc_list = ['Steppe', 'CHG', 'EHG', 'WHG', 'Farmer', 'African']

exclude_mhc = False
only_MHC = False

results_df = pd.DataFrame(columns=['phenotype', 'ancestry', 'PRS'])
for phen in phen_list:
    df1 = df[df[1]==phen]
    dict_of_PRS = dict()
    maf_df_all = pd.DataFrame()
    for n,anc in enumerate(anc_list):
        df2 = df1.loc[(df1[2]==anc)&(df1[8]==0)]
        maf_list = list()
        beta_list = list()
        position_list = list()
        chr_list = list()
        for i in range(1,23):
            try:
                maf_list += df2.loc[(df2[3]==i)][10].item()
                beta_list += df2.loc[(df2[3]==i)][11].item()
                position_list += [int(x) for x in df2.loc[(df2[3]==i)][12].item()]
                chr_list += [str(i) for x in df2.loc[(df2[3]==i)][12].item()]
            except ValueError:
                continue
        maf_df_all[anc] = maf_list
        if n == len(anc_list)-1:
            maf_df_all['beta'] = beta_list
            maf_df_all['pos'] = position_list
            maf_df_all['chr'] = chr_list
        if exclude_mhc:
            maf_beta = pd.DataFrame([maf_list, beta_list, chr_list, position_list]).T
            maf_beta.columns = ['maf', 'beta', 'chr', 'pos']
            maf_beta = maf_beta.loc[~((maf_beta['pos']>28477797) & (maf_beta['pos']<33448354) & (maf_beta['chr']=='6'))]
        elif only_MHC:
            maf_beta = pd.DataFrame([maf_list, beta_list, chr_list, position_list]).T
            maf_beta.columns = ['maf', 'beta', 'chr', 'pos']
            maf_beta = maf_beta.loc[(maf_beta['pos']>28477797) & (maf_beta['pos']<33448354) & (maf_beta['chr']=='6')]
        else:
            maf_beta = pd.DataFrame([maf_list, beta_list]).T
            maf_beta.columns = ['maf', 'beta']
        maf_beta['meangen'] = maf_beta['maf']*maf_beta['beta']
        dict_of_PRS[anc] = maf_beta['meangen'].sum()
    if exclude_mhc:
        maf_df_all = maf_df_all.loc[~((maf_df_all['pos']>28477797) & (maf_df_all['pos']<33448354) & (maf_df_all['chr']=='6'))]
    elif only_MHC:
        maf_df_all = maf_df_all.loc[(maf_df_all['pos']>28477797) & (maf_df_all['pos']<33448354) & (maf_df_all['chr']=='6')]
    maf_df_all['mean_freq'] = maf_df_all[anc_list].mean(axis=1)
    maf_df_all['score'] = maf_df_all['mean_freq']*(1-maf_df_all['mean_freq'])*(maf_df_all['beta']**2)

    mean_PRS = statistics.mean(list(dict_of_PRS.values()))
    varmean = maf_df_all['score'].sum()

    for anc in anc_list:
#     for anc in ['Farmer', 'Steppe', 'African', 'EHG', 'CHG', 'WHG']: # Can order results here if desired
        meangen = dict_of_PRS[anc] - mean_PRS
        meangenvec = 2 * meangen / (math.sqrt(4*varmean))
        results_df = results_df.append({'phenotype':phen, 'ancestry': anc, 'PRS': meangenvec}, ignore_index=True)
theta = results_df # For accelerated bootstrap below
   

In [ ]:
# Bootstrap loci, and plot means with SD
df = pd.read_csv("ms_results/PRS_calculations_v4_discovery_0.7_nobootstrap", sep=" ", header=None, converters={9: literal_eval, 10: literal_eval, 11: literal_eval, 12: literal_eval}) 
phen_list = ['phenotype']
anc_list = df[2].unique().tolist()

exclude_mhc = False
only_MHC = False
boostrap_number = 1000
anc_list = ['African', 'WHG', 'EHG', 'CHG', 'Yamnaya','Farmer','EastAsian']
anc_list = ['Steppe', 'CHG', 'EHG', 'WHG', 'Farmer', 'African']

results_df = pd.DataFrame(columns=['phenotype', 'ancestry', 'PRS'])
for phen in phen_list:
    beta_list = list()
    position_list = list()
    chr_list = list()                             
    for i in range(1,23):
        try:
            beta_list += df.loc[(df[2]=='Farmer')&(df[8]==0)&(df[1]==phen)&(df[3]==i)][11].item()
            position_list += df.loc[(df[2]=='Farmer')&(df[8]==0)&(df[1]==phen)&(df[3]==i)][12].item()
            for x in df.loc[(df[2]=='Farmer')&(df[8]==0)&(df[1]==phen)&(df[3]==i)][11].item():
                chr_list.append(df.loc[(df[2]=='Farmer')&(df[8]==0)&(df[1]==phen)&(df[3]==i)][3].item())
        except ValueError:
            continue
    beta_position_map = pd.DataFrame([beta_list, position_list, chr_list]).T
    beta_position_map.columns = ['beta', 'pos', 'chr']
    print(phen + " has " + str(len(beta_list)) + " SNPs")
    if exclude_mhc:
        beta_position_map_excluded = beta_position_map.loc[~((beta_position_map['pos'].astype(int)>28477797) & (beta_position_map['pos'].astype(int)<33448354) & (beta_position_map['chr']==6))]
        print("Excluding MHC SNPs, now " + str(beta_position_map_excluded.shape[0]) + " SNPs")
    elif only_MHC:
        beta_position_map_excluded = beta_position_map.loc[(beta_position_map['pos'].astype(int)>28477797) & (beta_position_map['pos'].astype(int)<33448354) & (beta_position_map['chr']==6)]
        print("Only including MHC SNPs, now " + str(beta_position_map_excluded.shape[0]) + " SNPs")
    df_beta = pd.DataFrame(beta_list)
    for bs in range(boostrap_number):
        maf_beta=pd.DataFrame()
        if exclude_mhc:
            bootstrap = beta_position_map_excluded.sample(n=beta_position_map_excluded.shape[0], replace=True)
        elif only_MHC:
            bootstrap = beta_position_map_excluded.sample(n=beta_position_map_excluded.shape[0], replace=True)
        else:
            bootstrap = beta_position_map[['beta', 'pos', 'chr']].sample(n=beta_position_map.shape[0], replace=True)
        dict_of_PRS = dict()
        maf_df_all = pd.DataFrame()
        for anc in anc_list:
            df1 = df.loc[(df[1]==phen)&(df[8]==0)&(df[2]==anc)]
            maf_list = list()
            for i in range(1,23):
                try:
                    maf_list += df1.loc[(df1[3]==i)][10].item()
                except ValueError:
                    continue
            beta_position_map['maf'] = maf_list
            maf_beta = pd.merge(bootstrap, beta_position_map, )
            maf_beta['meangen'] = maf_beta['maf']*maf_beta['beta']
            dict_of_PRS[anc] = maf_beta['meangen'].sum()
            maf_df_all[anc] = maf_beta['maf']
        maf_df_all['beta'] = maf_beta['beta']
        maf_df_all['mean_freq'] = maf_df_all[anc_list].mean(axis=1)
        maf_df_all['score'] = maf_df_all['mean_freq']*(1-maf_df_all['mean_freq'])*(maf_df_all['beta']**2)
        mean_PRS = statistics.mean(list(dict_of_PRS.values()))
        varmean = maf_df_all['score'].sum()

        for anc in anc_list:
            meangen = dict_of_PRS[anc] - mean_PRS
            meangenvec = 2 * meangen / (math.sqrt(4*varmean))
            results_df = results_df.append({'phenotype':phen, 'ancestry': anc, 'PRS': meangenvec}, ignore_index=True)


In [ ]:
# Find errors using bootstrap results directly (i.e. not assuming normal distribution)

ax = sns.violinplot(x="ancestry", y="PRS", data=results_df, inner='box')
ax.axhline(y=0)

empirical_ci_list = list()
# for anc in ['EastAsian', 'CHG', 'EHG', 'WHG', 'Farmer', 'Yamnaya', 'African']:
for anc in ['EHG', 'Steppe', 'CHG', 'Farmer', 'WHG', 'African']:
    temp = results_df.loc[results_df['ancestry']==anc]
    sorted_estimates = np.sort(np.array(temp['PRS']))
    mean = sorted_estimates[int(0.5 * boostrap_number)]
    conf_interval = [sorted_estimates[int(0.025 * boostrap_number)], sorted_estimates[int(0.975 * boostrap_number)]]
    conf_interval_2 = [abs(x - mean) for x in conf_interval]
    empirical_ci_list.append([anc, mean, conf_interval, conf_interval_2])
empirical_ci = pd.DataFrame(empirical_ci_list, columns=['ancestry', 'mean', 'CI','CI2'])

g = sns.FacetGrid(empirical_ci, hue='ancestry', height=4, aspect=4, margin_titles = False)
g.map(plt.errorbar, 'ancestry', 'mean', 'CI2', marker="o", )
for ax in g.axes.flat:
    ax.set_title(ax.get_title(), fontsize='xx-large')
    ax.axhline(y=0 )
    

In [ ]:
#  Run accelerated bootstrap 
def calculate_PRS_anc(data, ancestry):
        results_df = pd.DataFrame(columns=['phenotype', 'ancestry', 'PRS'])
        maf_beta = pd.DataFrame()
        bootstrap = data
        dict_of_PRS = dict()
        maf_df_all = pd.DataFrame()
        for anc in anc_list:
            df1 = df.loc[(df[1]==phen)&(df[8]==0)&(df[2]==anc)]
            maf_list = list()
            for i in range(1,23):
                try:
                    maf_list += df1.loc[(df1[3]==i)][9].item()
                except ValueError:
                    continue
            beta_position_map['maf'] = maf_list
            maf_beta = pd.merge(bootstrap, beta_position_map)
            maf_beta['meangen'] = maf_beta['maf']*maf_beta['beta']
            dict_of_PRS[anc] = maf_beta['meangen'].sum()
            maf_df_all[anc] = maf_beta['maf']
        maf_df_all['beta'] = maf_beta['beta']
        maf_df_all['mean_freq'] = maf_df_all[anc_list].mean(axis=1)
        maf_df_all['score'] = maf_df_all['mean_freq']*(1-maf_df_all['mean_freq'])*(maf_df_all['beta']**2)
        mean_PRS = statistics.mean(list(dict_of_PRS.values()))
        varmean = maf_df_all['score'].sum()
        for anc in anc_list:
            meangen = dict_of_PRS[anc] - mean_PRS
            meangenvec = 2 * meangen / (math.sqrt(4*varmean))
            results_df = results_df.append({'phenotype':phen, 'ancestry': anc, 'PRS': meangenvec}, ignore_index=True)
        return results_df.loc[results_df['ancestry']==ancestry]['PRS'].item()
    
exclude_mhc = False
only_MHC = False

accelerated_ci_list = list()
for ancestry in anc_list:
    theta_hat = theta.loc[theta['ancestry'] == ancestry]['PRS'].item()
    I_list = []
    if exclude_mhc:
        n = beta_position_map_excluded  # exclude MHC
    elif only_MHC:
        n = beta_position_map_excluded  # only MHC
    else:
        n = beta_position_map  # include MHC
    n.reset_index(drop=True, inplace=True)
    for i in range(n.shape[0]): 
        n_len = n.shape[0]
        temp = n.iloc[n.index != i]
        theta_jack = calculate_PRS_anc(temp, ancestry)
        if theta_jack == float('+inf') or theta_jack == float('-inf'):
            print(f"* ERROR for {ancestry}, where theta_jack = {theta_jack} and i = {i}, skipping*")
            continue
        I_list.append((n_len - 1) * (theta_hat - theta_jack))
    I = np.array(I_list)
    a_hat = sum(I**3)/sum(I**2)**1.5/6
    # NB Make sure bootstrap loci cell above has been run to obtain results_df
    theta_boot = results_df.loc[results_df['ancestry'] == ancestry]['PRS'].to_numpy()
    newArr = theta_boot <= theta_hat
    newArr = newArr.astype(int)
    z0 = sp.norm.ppf(np.mean(newArr))
    a = a_hat
#     zu = np.array([sp.norm.ppf(0.085),sp.norm.ppf(0.915)])  # 83% confidence intervals 
    zu = np.array([sp.norm.ppf(0.025),sp.norm.ppf(0.975)])  # 95% confidence intervals 
    u_adjusted = sp.norm.cdf(z0 + (z0+zu)/(1-a*(z0+zu)))
    conf_interval = np.quantile(theta_boot, u_adjusted)
    conf_interval_2 = [abs(x - theta_hat) for x in conf_interval]
    accelerated_ci_list.append([ancestry, theta_hat, conf_interval, conf_interval_2])
accelerated_ci = pd.DataFrame(accelerated_ci_list, columns=['ancestry', 'mean', 'CI', 'CI2'])


In [ ]:
# Plot accelerated bootstrap 

accelerated_ci[['c1', 'c2']] = pd.DataFrame(accelerated_ci.CI2.tolist(), index= accelerated_ci.index)
asymmetric_error = [accelerated_ci['c1'].to_numpy(), accelerated_ci['c2'].to_numpy()]
accelerated_ci.iloc[5,0] = 'Outgroup'

fig, ax = plt.subplots()
plt.errorbar(accelerated_ci['ancestry'], accelerated_ci['mean'], asymmetric_error, fmt='o', color='red',
             ecolor='lightgray', elinewidth=3, capsize=0)
fig.set_size_inches(5,5)
ax.axhline(y=0 )
ax.set_ylim([-2.5, 3.50])

# plt.title("Rheumatoid Arthritis (HLA, SNPs)")
# plt.xlabel("Ancestry")
# plt.ylabel("Ancestral Risk Score")
# plt.savefig("ARS_plots/ra_HLA.png")



In [ ]:
# Plot mean and SD using bootstrapped individuals 
df = pd.read_csv("ms_results/PRS_calculations_v4_discovery_0.7_bootstrap", sep=" ", header=None, converters={9: literal_eval, 10: literal_eval, 11: literal_eval, 12: literal_eval}) 
phen_list = ['phenotype']
anc_list = df[2].unique().tolist()
anc_list = ['African', 'WHG', 'EHG', 'CHG', 'Yamnaya','Farmer','EastAsian']
anc_list = ['Steppe', 'CHG', 'EHG', 'WHG', 'Farmer', 'African']
boostrap_number = 1000

results_df = pd.DataFrame(columns=['phenotype', 'ancestry', 'PRS'])
for phen in phen_list:
    beta_list = list()
    for i in range(1,23):
        try:
            beta_list += df.loc[(df[2]=='Farmer')&(df[8]==0)&(df[1]==phen)&(df[3]==i)][11].item()
        except ValueError:
            continue
    if len(beta_list) < 10:
        print("Warning: " + phen + " has " + str(len(beta_list)) + " SNPs")
    if not beta_list:
        continue
    df_beta = pd.DataFrame(beta_list)
    for bs in range(0,boostrap_number):
        df1 = df[(df[1]==phen)&(df[8]==bs)]
        dict_of_PRS = dict()
        maf_df_all = pd.DataFrame()
        for anc in anc_list:
            df2 = df1.loc[df1[2]==anc]
            maf_list = list()
            for i in range(1,23):
                try:
                    maf_list += df2.loc[(df2[3]==i)][10].item()
                except ValueError:
                    continue
            maf_beta = pd.DataFrame([maf_list, beta_list]).T
            maf_beta.columns = ['maf', 'beta']
            maf_beta = pd.merge(df_beta, maf_beta, left_on=0, right_on='beta') # NB sometimes duplicates SNPs if there are more than 1 beta value the same
            maf_beta['meangen'] = maf_beta['maf']*maf_beta['beta']
            dict_of_PRS[anc] = maf_beta['meangen'].sum()
            maf_df_all[anc] = maf_beta['maf']
            
        maf_df_all['beta'] = maf_beta['beta']
        maf_df_all['mean_freq'] = maf_df_all[anc_list].mean(axis=1)
        maf_df_all['score'] = maf_df_all['mean_freq']*(1-maf_df_all['mean_freq'])*(maf_df_all['beta']**2)
        mean_PRS = statistics.mean(list(dict_of_PRS.values()))
        varmean = maf_df_all['score'].sum()

        for anc in anc_list:
            meangen = dict_of_PRS[anc] - mean_PRS
            meangenvec = 2 * meangen / (math.sqrt(4*varmean))
            results_df = results_df.append({'phenotype':phen, 'ancestry': anc, 'PRS': meangenvec}, ignore_index=True)

# Find errors using bootstrap results directly (i.e. not assuming normal distribution)
ax = sns.violinplot(x="ancestry", y="PRS", data=results_df, inner='box')
ax.axhline(y=0)
empirical_ci_list = list()
for anc in anc_list:
    temp = results_df.loc[results_df['ancestry']==anc]
    sorted_estimates = np.sort(np.array(temp['PRS']))
    mean = sorted_estimates[int(0.5 * boostrap_number)]
    conf_interval = [sorted_estimates[int(0.025 * boostrap_number)], sorted_estimates[int(0.975 * boostrap_number)]]
    conf_interval_2 = [abs(x - mean) for x in conf_interval]
    empirical_ci_list.append([anc, mean, conf_interval, conf_interval_2])
empirical_ci = pd.DataFrame(empirical_ci_list, columns=['Ancestry', 'Ancestral Risk Score', 'CI','CI2'])

g = sns.FacetGrid(empirical_ci, hue='Ancestry', height=5, 
                  aspect=1, margin_titles = False)
g.map(plt.errorbar, 'Ancestry', 'Ancestral Risk Score', 'CI2', fmt='o', color='red',
             ecolor='lightgray', elinewidth=3, capsize=0)
for ax in g.axes.flat:
    ax.set_title("Multiple Sclerosis (all, individuals)")
    ax.set_ylim([-2, 2])
    ax.axhline(y=0 )


In [ ]:
# Find errors using bootstrap results directly (i.e. not assuming normal distribution)
ax = sns.violinplot(x="ancestry", y="PRS", data=results_df, inner='box')
ax.axhline(y=0)

empirical_ci_list = list()
for anc in anc_list:
    temp = results_df.loc[results_df['ancestry']==anc]
    sorted_estimates = np.sort(np.array(temp['PRS']))
    mean = sorted_estimates[int(0.5 * boostrap_number)]
    conf_interval = [sorted_estimates[int(0.025 * boostrap_number)], sorted_estimates[int(0.975 * boostrap_number)]]
    conf_interval_2 = [abs(x - mean) for x in conf_interval]
    empirical_ci_list.append([anc, mean, conf_interval, conf_interval_2])
empirical_ci = pd.DataFrame(empirical_ci_list, columns=['Ancestry', 'Ancestral Risk Score', 'CI','CI2'])
display(empirical_ci)

g = sns.FacetGrid(empirical_ci, hue='Ancestry', height=5, 
                  aspect=1, margin_titles = False)
g.map(plt.errorbar, 'Ancestry', 'Ancestral Risk Score', 'CI2', fmt='o', color='red',
             ecolor='lightgray', elinewidth=3, capsize=0)
for ax in g.axes.flat:
    ax.set_title("Multiple Sclerosis (all, individuals)")
    ax.set_ylim([-2, 2])
    ax.axhline(y=0 )